In [1]:
import json
import heapq
import re
from datetime import datetime


DB_FILENAME = 'finalvolunteer_and_organization_data.json'

# Load the database from a file (if exists) or initialize an empty list
def load_database():
    try:
        with open(DB_FILENAME, 'r') as file:
            # Attempt to load JSON data, handling potential decoding errors
            try:
                return json.load(file)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from {DB_FILENAME}: {e}")
                print("Starting with an empty database.")
                return []  # Return an empty list if decoding fails
    except FileNotFoundError:
        print(f"{DB_FILENAME} not found. Starting with an empty database.")
        return []

# Save the database to a file
def save_database(db):
    # Create a custom JSON encoder to handle datetime objects
    class DateTimeEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, datetime):
                return obj.isoformat()  # Convert datetime to ISO format string
            return super().default(obj)

    with open(DB_FILENAME, 'w') as file:
        # Use the custom encoder when dumping the data
        json.dump(db, file, indent=4, cls=DateTimeEncoder)
    print(f"Database saved successfully at {DB_FILENAME}")

# Global Database
db = load_database()



In [2]:


# Helper functions to check if user exists
def find_user_by_email(email):
    return next((user for user in db if user["email"] == email), None)

def find_user_by_username(username):
    return next((user for user in db if user["username"] == username), None)

#To check the email is valid
def is_valid_email(email):
    # Regular expression to validate email format
    #the characters before the @ are: a - z, A - Z, 0 - 9, ., _, %, +, -
    #the characters after the @ and before the top level domain are: a - z, A - Z
    #the characters after the domain name (Top level domain): (2,) has at least 2 characters, are from a - z, A - Z
    email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(email_regex, email)

# Function for signing up a new user
def sign_up():
    email = input("Enter your email: ")
    user_name = input("Enter username: ")
    user_type = input("Are you a volunteer or an organization? (volunteer/organization): ").lower()
    password = input("Enter your password: ")
    confirm_password = input("Confirm your password: ")

    if not is_valid_email(email):
        print("Invalid email format. Please try again.")
        return sign_up()

    if password != confirm_password:
        print("Passwords do not match. Try again.")
        return sign_up()

    # Check if email already exists
    if find_user_by_email(email):
        print("Email already registered. Log in.")
        return log_in()

    if find_user_by_username(user_name):
        print("Username already taken. Try again.")
        return sign_up()


    # Initialize new user
    user = {
        "username": user_name,
        "email": email,
        "password": password,
        "user_type": user_type,
        "details": {}
    }

    # Input details based on user type
    if user_type == "volunteer":
        input_volunteer_info(user)
    elif user_type == "organization":
        input_organization_info(user)

    db.append(user)
    save_database(db)
    print("Sign-up successful!")
    return user

# Function for logging in an existing user
def log_in():
    email = input("Enter your email: ")
    user = find_user_by_email(email)

    if not user:
        print("Email not registered. Please sign up.")
        return sign_up()

    password = input("Enter your password: ")
    if user["password"] == password:
        print("Login successful!")
        return user
    else:
        print("Incorrect password. Try again.")
        return log_in()

# Function to input volunteer details
def input_volunteer_info(user):
  try:
        user['details']['name'] = input("Enter your name: ")

        # Process locations
        user['details']['location'] = [
            loc.lower().strip().replace(" ", "")
            for loc in input("Enter cities where you would like to collaborate (comma-separated): ").split(',')
        ]

        user['details']['availability'] = input("Enter availability type (individual events/for a period): ").lower().strip().replace(" ", "")

        # Validate start and end dates
        while True:
            try:
                user["details"]["start_date"] = datetime.strptime(input("Enter start date (YYYY-MM-DD): "), '%Y-%m-%d')
                user["details"]["end_date"] = datetime.strptime(input("Enter end date (YYYY-MM-DD): "), '%Y-%m-%d')
                if user["details"]["end_date"] < user["details"]["start_date"]:
                    print("End date cannot be before start date. Please try again.")
                else:
                    break
            except ValueError:
                print("Invalid date format. Please use YYYY-MM-DD.")

        # Process causes, skills, and languages
        user['details']['cause'] = [
            cause.lower().strip().replace(" ", "")
            for cause in input("Enter causes your want to address (comma-separated): (education,environmental,animal,humanitarian,health,youth) ").split(',')
        ]
        user['details']['required_skills'] = [
            skill.lower().strip().replace(" ", "")
            for skill in input("Enter skills  (comma-separated): (communication,teaching,administrative,social,health,teamwork) ").split(',')
        ]
        user['details']['languages'] = [
            lang.lower().strip().replace(" ", "")
            for lang in input("Enter languages spoken (comma-separated): ").split(',')
        ]


        # Initialize additional fields
        user['interested_organizations'] = []
        user['potential_organizations'] = []
        user['matches'] = []
        user['feedback'] = []

        print("Volunteer details saved successfully.")
        save_database(db)

  except Exception as e:
        print(f"An error occurred while inputting organization details: {e}")

#Function to check if the availability for a period matches
def check_period_dates_overlap(volunteer, organization): #cambiar formato cuando el usuario rellene
      vol_start = volunteer["details"]['start_date'].date() if isinstance(volunteer["details"]['start_date'], datetime) else datetime.fromisoformat(volunteer["details"]['start_date']).date()
      vol_end = volunteer["details"]['end_date'].date() if isinstance(volunteer["details"]['end_date'], datetime) else datetime.fromisoformat(volunteer["details"]['end_date']).date()
      org_start = organization["details"]['start_date'].date() if isinstance(organization["details"]['start_date'], datetime) else datetime.fromisoformat(organization["details"]['start_date']).date()
      org_end = organization["details"]['end_date'].date() if isinstance(organization["details"]['end_date'], datetime) else datetime.fromisoformat(organization["details"]['end_date']).date()
      if volunteer["details"]["availability"] != organization["details"]["availability"]:
        return False
      else:
        return vol_start <= org_end and vol_end >= org_start

#Function to input organizations details
def input_organization_info(user):
  try:
        user['details']['name'] = input("Enter your name: ")

        # Process locations
        user['details']['location'] = [
            loc.lower().strip().replace(" ", "")
            for loc in input("Enter cities where you operate(comma-separated): ").split(',')
        ]

        user['details']['availability'] = input("Enter availability type (individual events/for a period): ").lower().strip().replace(" ","")

        # Validate start and end dates
        while True:
            try:
                start_date = input("Enter start date (YYYY-MM-DD): ")
                end_date = input("Enter end date (YYYY-MM-DD): ")
                user["details"]["start_date"] = datetime.strptime(start_date, '%Y-%m-%d')
                user["details"]["end_date"] = datetime.strptime(end_date, '%Y-%m-%d')
                if user["details"]["end_date"] < user["details"]["start_date"]:
                    print("End date cannot be before start date. Please try again.")
                else:
                    break
            except ValueError:
                print("Invalid date format. Please use YYYY-MM-DD.")

        # Process causes, skills, and languages
        user['details']['cause'] = [
            cause.lower().strip().replace(" ", "")
            for cause in input("Enter causes your organization addresses (comma-separated): (education,environmental,animal,humanitarian,health,youth) ").split(',')
        ]
        user['details']['required_skills'] = [
            skill.lower().strip().replace(" ", "")
            for skill in input("Enter skills required (comma-separated): (communication,teaching,administrative,social,health,teamwork) ").split(',')
        ]
        user['details']['languages'] = [
            lang.lower().strip().replace(" ", "")
            for lang in input("Enter languages required (comma-separated): ").split(',')
        ]

        user['details']['description'] = input("Provide a brief description of your organization: ")

        # Initialize additional fields
        user['interested_volunteers'] = []
        user['potential_volunteers'] = []
        user['matches'] = []
        user['feedback'] = []

        print("Organization details saved successfully.")
        save_database(db)

  except Exception as e:
        print(f"An error occurred while inputting organization details: {e}")

# Function to calculate compatibility score between volunteer and organization
def calculate_score(volunteer, organization):
    score = 0
    section_scores = {}

    # Causes (30%)
    common_causes = len(set(volunteer['details']['cause']) & set(organization['details']['cause']))
    section_scores['cause'] = (common_causes / len(organization['details']['cause']) * 100) if organization['details']['cause'] else 0
    score += section_scores['cause'] * 0.3

    # Skills (20%)
    common_skills = len(set(volunteer['details']['required_skills']) & set(organization['details']['required_skills']))
    section_scores['skills'] = (common_skills / len(organization['details']['required_skills']) * 100) if organization['details']['required_skills'] else 0
    score += section_scores['skills'] * 0.2

    # Languages (20%)
    common_languages = len(set(volunteer['details']['languages']) & set(organization['details']['languages']))
    section_scores['languages'] = (common_languages / len(organization['details']['languages']) * 100) if organization['details']['languages'] else 0
    score += section_scores['languages'] * 0.2

    # Location (30%)
    common_locations = len(set(volunteer['details']['location']) & set(organization['details']['location']))
    section_scores['location'] = (common_locations / len(organization['details']['location']) * 100) if organization['details']['location'] else 0
    score += section_scores['location'] * 0.3

    return round(score, 2), section_scores

# Function to find matches for a volunteer
def find_matches(volunteer):
    matches = []
    for organization in db:
        if organization['user_type'] == 'organization':
          if check_period_dates_overlap(volunteer,organization):
            score, section_scores = calculate_score(volunteer, organization)
            if score > 30:  # Only consider matches with a score above 30%
                matches.append({
                    "username": organization['username'],
                    "details": organization['details'],
                    "score": score,
                    "section_scores": section_scores,
                    "feedback": organization['feedback'],
                    "interested_volunteers" : organization['interested_volunteers'],
                    'potential_volunteers': organization["potential_volunteers"],
                    'matches' : organization["matches"]
                })
    return matches

# Function to find matches for an organization
def find_matches_for_org(organization):
    matches = []
    for volunteer in db:
        if volunteer['user_type'] == 'volunteer':
          if check_period_dates_overlap(volunteer,organization):
            score, section_scores = calculate_score(volunteer, organization)
            if score > 30:  # Only consider matches with a score above 30%
                matches.append({
                    "username": volunteer['username'],
                    "details": volunteer['details'],
                    "score": score,
                    "section_scores": section_scores,
                    "feedback": volunteer['feedback'],
                    "interested_organizations" : volunteer['interested_organizations'],
                    "potential_organizations": volunteer["potential_organizations"],
                    "matches": volunteer["matches"]
                })
    return matches

#Function to create the priority queue
def view_priority_matches(volunteer):
    matches = find_matches(volunteer)

    # Use a priority queue (max-heap) for matches based on scores
    priority_queue = [(-match['score'], idx, match) for idx, match in enumerate(matches)]
    heapq.heapify(priority_queue)

    if not priority_queue:
        print("No suitable matches found.")
        return

    print("\nViewing Matches:")
    while priority_queue:
        # Extract the top match from the priority queue
        score,idx, match = heapq.heappop(priority_queue)

        print(f"\nTop Match: {match['username']} - Score: {-score}")
        print("""
        \n1. View basic details
        \n2. View compatibility analysis
        \n3. View feedback
        \n4. Apply to this organization
        \n5. Skip to the next match
        \n6. Exit\n""")

        choice = input("Enter your choice (1-6): ")
        while True:
          if choice == '1':
              for key in ["name", "location", "cause", "required_skills", "languages"]:
                  print(f"{key.capitalize()}: {match['details'][key]}")
          elif choice == '2':
              for category, score in match["section_scores"].items():
                  print(f"{category.capitalize()}: {score:.2f}%")
          elif choice == '3':
              if match['feedback']:
                  for feedback in match['feedback']:
                      print(f"Feedback: {feedback}")
              else:
                  print("No feedback available.")
              if "ratings" in match and match["ratings"]:
                  for rating in match["ratings"]:
                      print(f"Rating: {rating}")
          elif choice == '4':
              apply_to_organization(volunteer, match)
              break  # End interaction after applying
          elif choice == '5':
              print("Skipping to the next match.")
              break
          elif choice == '6':
              print("Exiting the matches view.")
              return
          else:
              print("Invalid choice. Try again.")

          print("""
          \n1. View basic details
          \n2. View compatibility analysis
          \n3. View feedback
          \n4. Apply to this organization
          \n5. Skip to the next match
          \n6. Exit\n""")
          choice = input("Enter your choice (1-6): ")

        # If the queue is empty after skipping
        if not priority_queue:
            print("\nNo more matches available.")

#Function to create the priority queue for organizations
def view_priority_matches_for_org(organization):
    matches = find_matches_for_org(organization)

    # Use a priority queue (max-heap) for matches based on scores
    priority_queue = [(-match['score'], idx, match) for idx, match in enumerate(matches)]
    heapq.heapify(priority_queue)  # Turn the list into a valid heap

    if not priority_queue:
        print("No suitable matches found.")
        return

    print("\nViewing Matches:")
    while priority_queue:
        # Extract the top match from the priority queue
        score, idx, match = heapq.heappop(priority_queue)

        print(f"\nTop Match: {match['username']} - Score: {-score}")
        print("""
        \n1. View basic details
        \n2. View compatibility analysis
        \n3. View feedback
        \n4. Apply to this organization
        \n5. Skip to the next match
        \n6. Exit\n""")

        while True:
            choice = input("Enter your choice (1-6):").strip()
            if choice == "6":
                print("Exiting the matches view.")
                return
            elif choice == '1':
                for key in ["name", "location", "cause", "required_skills", "languages"]:
                    print(f"{key.capitalize()}: {match['details'][key]}")
            elif choice == '2':
                for category, score in match["section_scores"].items():
                    print(f"{category.capitalize()}: {score:.2f}%")
            elif choice == '3':
                if match['feedback']:
                    for feedback in match['feedback']:
                        print(f"Feedback: {feedback}")
                else:
                    print("No feedback available.")
                if "ratings" in match and match["ratings"]:
                    for rating in match["ratings"]:
                        print(f"Rating: {rating}")

            elif choice == '4':
                apply_to_volunteers(organization, match)
                break  # End interaction after applying
            elif choice == '5':
                print("Skipping to the next match.")
                break  # Exit the inner loop to the next match
            else:
                print("Invalid choice. Try again.")

        # If the queue is empty after skipping to the next match
        if not priority_queue:
            print("\nNo more matches available.")


#Function to apply to organisations
def apply_to_organization(volunteer,organization):
    if organization['username'] not in volunteer['potential_organizations'] and organization['username'] not in volunteer['matches']:
        if volunteer['username'] not in organization['potential_volunteers']:
            organization['interested_volunteers'].append(volunteer['username'])
            volunteer['potential_organizations'].append(organization['username'])

        else:
          organization["matches"].append(volunteer["username"])
          volunteer["matches"].append(organization["username"])
          organization["potential_volunteers"].remove(volunteer["username"])

        print("Application submitted successfully!")
        save_database(db)

    else:
      print("Already applied to this organization")

#Function to apply to volunteers
def apply_to_volunteers(organization,volunteer):
    if volunteer['username'] not in organization['potential_volunteers'] and volunteer['username'] not in organization['matches']:
        if organization['username'] not in volunteer['potential_organizations']:
            organization['potential_volunteers'].append(volunteer['username'])
            volunteer['interested_organizations'].append(organization['username'])

        else:
          organization["matches"].append(volunteer["username"])
          volunteer["matches"].append(organization["username"])
          volunteer["potential_organizations"].remove(organization["username"])

        print("Application submitted successfully!")
        save_database(db)

    else:
      print("Already applied to this volunteer")

# Main function to drive the application
def possible_matches(user):
    if user and user['user_type'] == 'volunteer':
        view_priority_matches(user)

    elif user and user['user_type'] == 'organization':
        view_priority_matches_for_org(user)
def see_lists(user):
    if user and user['user_type'] == 'volunteer':
        print(f"Interested organizations: {user['interested_organizations']}\n")
        print(f"Potential organizations: {user['potential_organizations']}\n")
        print(f"Matches: {user['matches']}\n")
    elif user and user['user_type'] == 'organization':
        print(f"Interested volunteers: {user['interested_volunteers']}\n")
        print(f"Potential volunteers: {user['potential_volunteers']}\n")
        print(f"Matches: {user['matches']}\n")

def provide_feedback(user):
    if user["user_type"] == "volunteer":
        matches = user["matches"]
        feedback_target = "organization"
    elif user["user_type"] == "organization":
        matches = user["matches"]
        feedback_target = "volunteer"
    else:
        print("Invalid user type for feedback.")
        return

    if not matches:
        print(f"No matches available for feedback.")
        return

    response = input(f"Would you like to provide feedback for your matched {feedback_target}s? (yes/no): ").strip().lower()

    if response != "yes":
        print("No feedback provided.")
        return

    for match_username in matches:
        match = find_user_by_username(match_username)
        if not match:
            print(f"Match {match_username} not found in the database.")
            continue

        feedback = input(f"Write feedback for {match['username']}: ").strip()
        if feedback:
                match["feedback"].append(f"From {user['username']}: {feedback}")
                print(f"Feedback for {match['username']} saved!")

    while True:
      rating = input(f"Rate {match['username']} from 0 to 5 (or press Enter to skip): ").strip()

      if not rating:  # Allow skipping ratings
          break

      if rating.replace('.', '', 1).isdigit():  # Check if input is numeric (allows decimals)
          rating = float(rating)  # Convert to float
          if 0 <= rating <= 5:
              match.setdefault("ratings", []).append({"from": user["username"], "rating": rating})
              print(f"Rating for {match['username']} saved!")
              break
          else:
              print("Please enter a valid rating between 0 and 5.")  # Out of range
      else:
          print("Invalid input. Please enter a number between 0 and 5.")  # Non-numeric input

      save_database(db)

def delete_from_database(db, username_to_delete):
    # Filtrar los elementos que no coinciden con el username a eliminar
    updated_db = [item for item in db if item.get('username') != username_to_delete]

    # Verificar si se eliminó algo
    if len(updated_db) < len(db):
        print(f"User '{username_to_delete}' deleted successfully.")
    else:
        print(f"User '{username_to_delete}' not found in the database.")
    db = updated_db

def main():
    print("Welcome to Volunteer Match!")
    user = None
    while not user:
        choice = input("Do you want to log in or sign up? (login/signup/exit): ").lower()
        if choice == 'login':
            user = log_in()
        elif choice == 'signup':
            user = sign_up()
        elif choice == 'exit':
            print("Exiting the application.")
            return
        else:
            print("Invalid choice.")

    print("""
    \n1. See possible matches
    \n2. See interested and potential users and matches
    \n3. Provide feedback
    \n4. Delete account
    \n5. Exit\n""")

    selection = input("What do you want to do?:")
    while selection != "5":
        if selection == "1":
          possible_matches(user)
        elif selection == "2":
          see_lists(user)
        elif selection == "3":
          provide_feedback(user)
        elif selection == "4":
          checker = input("Are you sure? (yes/no)")
          if checker == "yes":
            username_to_delete = user["username"]
            db = delete_from_database(db, username_to_delete)
            save_database(db)
        else:
          print("Invalid choice.")
        print("""
        \n1. See possible matches
        \n2. See interested and potential users and matches
        \n3. Provide feedback
        \n4. Delete account
        \n5. Exit\n""")
        selection =  input("What do you want to do?:")










In [ ]:
main()